# Carga, configuración e inicio de proyecto

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import time
from torch.amp import autocast, GradScaler
from pathlib import Path
import numpy as np
from collections import defaultdict

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = (64, 64)
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 3e-4
TEMPERATURE = 0.07

# Configuración SSL
USE_SUBSET = True
SUBSET_SUBJECTS = 30
SUBSET_CONDITIONS = ['nm-01', 'nm-02', 'nm-03', 'nm-04']
SUBSET_ANGLES = ['090', '180']
SUBSET_FRAMES_PER_SEQ = 15

print(f"Device: {DEVICE}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Device: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


# SSL Phase

In [10]:
class CASIAB_SSL(Dataset):
    """
    Data structure: subject_id/condition(bg-01,nm-01,...)/angle(000,018,...)/frames
    """
    def __init__(self, root_path, use_subset=True, subset_subjects=10, subset_conditions=None, subset_angles=None, frames_per_seq=15):
        self.root = Path(root_path)
        self.images = []
        
        # Obtener sujetos
        all_subjects = sorted([d for d in self.root.iterdir() if d.is_dir()])
        subjects = all_subjects[:subset_subjects] if use_subset else all_subjects
        
        print(f"\nDataset desde: {root_path}")
        print(f"Sujetos: {len(subjects)}/{len(all_subjects)}")
        if subset_conditions:
            print(f"Condiciones: {subset_conditions}")
        if subset_angles:
            print(f"Ángulos: {subset_angles}")
        
        total_sequences = 0
        for subject_dir in subjects:
            for condition_dir in subject_dir.iterdir():
                if not condition_dir.is_dir():
                    continue
                
                # Filtrar condiciones
                if subset_conditions and condition_dir.name not in subset_conditions:
                    continue
                    
                for angle_dir in condition_dir.iterdir():
                    if not angle_dir.is_dir():
                        continue
                    
                    # Filtrar ángulos 
                    if subset_angles and angle_dir.name not in subset_angles:
                        continue
                    
                    total_sequences += 1
                    frames = sorted([f for f in angle_dir.iterdir() if f.suffix.lower() in ['.png', '.jpg', '.bmp']])
                    
                    # Muestreo uniforme de frames
                    if len(frames) > frames_per_seq:
                        step = len(frames) / frames_per_seq
                        frames = [frames[int(i * step)] for i in range(frames_per_seq)]
                    
                    self.images.extend(frames)
        
        print(f"Secuencias: {total_sequences}")
        print(f"Total frames: {len(self.images)}")
        
        # Transformaciones
        self.base_transform = transforms.Compose([
            transforms.Resize(IMG_SIZE, antialias=True),
            transforms.ToTensor(),
        ])
        
        self.ssl_transform = transforms.Compose([
            transforms.RandomResizedCrop(IMG_SIZE, scale=(0.5, 1.0), antialias=True),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(20),
            transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))], p=0.3),
        ])
    
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = Image.open(self.images[idx]).convert("L")
        img = self.base_transform(img)
        
        view1 = self.ssl_transform(img)
        view2 = self.ssl_transform(img)
        
        return view1, view2

In [4]:
class GaitBackbone(nn.Module):
    def __init__(self, embed_dim=256):
        super().__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Projection head for SimCLR
        self.projection = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embed_dim)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        z = self.projection(x)
        z = F.normalize(z, dim=1)
        return z

class NTXentLoss(nn.Module):
    """
    NT-Xent Loss (SimCLR).
    """
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(reduction="mean")
    
    def forward(self, z1, z2):
        batch_size = z1.size(0)
        
        # Forzar float32 para evitar overflow en mixed precision
        z1 = z1.float()
        z2 = z2.float()
        
        z = torch.cat([z1, z2], dim=0)
        sim_matrix = torch.mm(z, z.t()) / self.temperature
        mask = torch.eye(2 * batch_size, dtype=torch.bool, device=z.device)
        sim_matrix.masked_fill_(mask, -1e9)  # Reducido para evitar overflow
        
        labels = torch.cat([
            torch.arange(batch_size, 2 * batch_size),
            torch.arange(0, batch_size)
        ]).to(z.device)
        
        loss = self.criterion(sim_matrix, labels)
        return loss

## Execute SSL

In [5]:
def train_ssl(root_path, save_path="backbone_ssl_best.pth"):
    dataset = CASIAB_SSL(
        root_path=root_path,
        use_subset=USE_SUBSET,
        subset_subjects=SUBSET_SUBJECTS,
        subset_conditions=SUBSET_CONDITIONS,
        subset_angles=SUBSET_ANGLES,
        frames_per_seq=SUBSET_FRAMES_PER_SEQ
    )
    
    # No add multiprocessing (Windows)
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0,
        pin_memory=True if DEVICE == "cuda" else False,
    )
    
    # Modelo
    model = GaitBackbone(embed_dim=256).to(DEVICE)
    criterion = NTXentLoss(temperature=TEMPERATURE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    
    # GradScaler para mixed precision (desactivado por defecto para estabilidad)
    USE_MIXED_PRECISION = False  # Cambiar a True si no hay problemas de overflow
    scaler = GradScaler('cuda') if (DEVICE == "cuda" and USE_MIXED_PRECISION) else None
    
    print(f"\nConfiguración:")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Epochs: {EPOCHS}")
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Steps/epoch: {len(loader)}")
    print(f"Total steps: {EPOCHS * len(loader)}\n")
    
    best_loss = float('inf')
    
    for epoch in range(1, EPOCHS + 1):
        model.train()
        epoch_loss = 0
        start_time = time.time()
        
        for batch_idx, (view1, view2) in enumerate(loader):
            view1 = view1.to(DEVICE, non_blocking=True)
            view2 = view2.to(DEVICE, non_blocking=True)
            
            if scaler:
                with autocast('cuda'):
                    z1 = model(view1)
                    z2 = model(view2)
                    loss = criterion(z1, z2)
                
                optimizer.zero_grad(set_to_none=True)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                z1 = model(view1)
                z2 = model(view2)
                loss = criterion(z1, z2)
                
                optimizer.zero_grad(set_to_none=True)
                loss.backward()
                optimizer.step()
            
            epoch_loss += loss.item()
        
        scheduler.step()
        
        avg_epoch_loss = epoch_loss / len(loader)
        epoch_time = time.time() - start_time
        
        print(f"Epoch {epoch:2d}/{EPOCHS} | Loss: {avg_epoch_loss:.4f} | "
              f"Time: {epoch_time:.1f}s | LR: {scheduler.get_last_lr()[0]:.6f}")
        
        if avg_epoch_loss < best_loss:
            best_loss = avg_epoch_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
            }, save_path)
    
    print(f"Entrenamiento finalizado")
    print(f"Mejor loss: {best_loss:.4f}")
    print(f"Modelo guardado: {save_path}")
    
    return model


In [11]:
ROOT_PATH = "C:/Users/JuanTF/Desktop/Gait_Recognition/archive/output"
    
if not os.path.exists(ROOT_PATH):
    print(f"ERROR: No existe {ROOT_PATH}")

model = train_ssl(ROOT_PATH, save_path="backbone_ssl_best.pth")


Dataset desde: C:/Users/JuanTF/Desktop/Gait_Recognition/archive/output
Sujetos: 30/124
Condiciones: ['nm-01', 'nm-02', 'nm-03', 'nm-04']
Ángulos: ['090', '180']
Secuencias: 240
Total frames: 3581

Configuración:
Batch size: 64
Epochs: 10
Learning rate: 0.0003
Steps/epoch: 56
Total steps: 560

Epoch  1/10 | Loss: 3.7324 | Time: 47.7s | LR: 0.000293
Epoch  2/10 | Loss: 2.6707 | Time: 9.4s | LR: 0.000271
Epoch  3/10 | Loss: 2.1379 | Time: 9.1s | LR: 0.000238
Epoch  4/10 | Loss: 1.8254 | Time: 9.1s | LR: 0.000197
Epoch  5/10 | Loss: 1.5906 | Time: 9.3s | LR: 0.000150
Epoch  6/10 | Loss: 1.4274 | Time: 9.4s | LR: 0.000104
Epoch  7/10 | Loss: 1.3103 | Time: 9.3s | LR: 0.000063
Epoch  8/10 | Loss: 1.2302 | Time: 9.4s | LR: 0.000030
Epoch  9/10 | Loss: 1.1376 | Time: 9.4s | LR: 0.000008
Epoch 10/10 | Loss: 1.0919 | Time: 9.3s | LR: 0.000001
Entrenamiento finalizado
Mejor loss: 1.0919
Modelo guardado: backbone_ssl_best.pth


#  Supervised Phase

In [13]:
ROOT_PATH = "C:/Users/JuanTF/Desktop/Gait_Recognition/archive/output"
SSL_CHECKPOINT = "backbone_ssl_best.pth"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = (64, 64)
BATCH_SIZE = 64
EPOCHS = 30
LEARNING_RATE = 1e-4  # Más bajo que SSL (fine-tuning)
MARGIN = 0.3  # Para triplet loss

EPOCHS_SUPERVISED = 30
EPOCHS_HYBRID = 30

# Subset
USE_SUBSET = True
SUBSET_TRAIN_SUBJECTS = 20  # De 74
SUBSET_VAL_SUBJECTS = 10    # De 25
SUBSET_TEST_SUBJECTS = 10   # De 25
SUBSET_FRAMES_PER_SEQ = 20

SUPERVISED_CONFIG = {
    'conditions': ['nm-01', 'nm-02'],
    'angles': ['090'],
    'train_range': (0, SUBSET_TRAIN_SUBJECTS if USE_SUBSET else 74),
    'val_range': (74, 74 + SUBSET_VAL_SUBJECTS if USE_SUBSET else 99),
    'test_range': (99, 99 + SUBSET_TEST_SUBJECTS if USE_SUBSET else 124),
}

if not os.path.exists(ROOT_PATH):
    raise FileNotFoundError(f"No existe: {ROOT_PATH}")

print(f"Subset mode: {USE_SUBSET}")
if USE_SUBSET:
    print(f"Train: {SUBSET_TRAIN_SUBJECTS} sujetos, Val: {SUBSET_VAL_SUBJECTS}, Test: {SUBSET_TEST_SUBJECTS}")

Subset mode: True
Train: 20 sujetos, Val: 10, Test: 10


In [ ]:
class CASIAB_Supervised(Dataset):
    def __init__(self, root_path, subject_range, conditions, angles=None, frames_per_seq=20, img_size=(64, 44), augment=False):
        self.root = Path(root_path)
        self.augment = augment
        self.samples = []
        self.subject_to_label = {}
        
        # Obtener person en el rango
        all_subjects = sorted([d.name for d in self.root.iterdir() if d.is_dir()])
        start_idx, end_idx = subject_range
        subjects = all_subjects[start_idx:end_idx]
        
        print(f"Sujetos: {start_idx+1:03d}-{end_idx:03d} ({len(subjects)} sujetos)")
        print(f"Condiciones: {conditions}")
        if angles:
            print(f"Ángulos: {angles}")
        
        # Mapeo id
        for label_id, subject_name in enumerate(subjects):
            self.subject_to_label[subject_name] = label_id
        
        # Cargar samples
        for subject_name in subjects:
            subject_dir = self.root / subject_name
            label_id = self.subject_to_label[subject_name]
            
            for condition_dir in subject_dir.iterdir():
                if not condition_dir.is_dir() or condition_dir.name not in conditions:
                    continue
                
                for angle_dir in condition_dir.iterdir():
                    if not angle_dir.is_dir():
                        continue
                    if angles and angle_dir.name not in angles:
                        continue
                    
                    frames = sorted([f for f in angle_dir.iterdir()
                                   if f.suffix.lower() in ['.png', '.jpg', '.bmp']])
                    
                    # Muestreo
                    if len(frames) > frames_per_seq:
                        step = len(frames) / frames_per_seq
                        frames = [frames[int(i * step)] for i in range(frames_per_seq)]
                    
                    for frame_path in frames:
                        self.samples.append({
                            'path': frame_path,
                            'label': label_id,
                            'subject': subject_name,
                            'condition': condition_dir.name,
                            'angle': angle_dir.name
                        })
        
        print(f"  Samples: {len(self.samples)}, Clases: {len(self.subject_to_label)}")
        
        # Transformaciones
        if augment:
            self.transform = transforms.Compose([
                transforms.Resize(img_size, antialias=True),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(10),
                transforms.ToTensor(),
                transforms.RandomErasing(p=0.3, scale=(0.02, 0.1)),
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize(img_size, antialias=True),
                transforms.ToTensor(),
            ])
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        img = Image.open(sample['path']).convert("L")
        img = self.transform(img)
        return img, sample['label']
    
    def get_num_classes(self):
        return len(self.subject_to_label)
    
    def get_sample_info(self, idx):
        """Retorna info del sample (útil para debugging)"""
        return self.samples[idx]

In [ ]:
# Already defined
class SSLBackbone(nn.Module):
    def __init__(self, embed_dim=256):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.projection = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embed_dim)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        z = self.projection(x)
        z = F.normalize(z, dim=1)
        return z

class SupervisedReIDModel(nn.Module):
    def __init__(self, backbone, num_classes, freeze_backbone=False):
        super().__init__()
        
        self.encoder = nn.Sequential(
            backbone.conv1,
            backbone.conv2,
            backbone.conv3,
            backbone.pool
        )
        
        if freeze_backbone:
            for param in self.encoder.parameters():
                param.requires_grad = False
            print("  ✓ Backbone congelado")
        
        self.bn = nn.BatchNorm1d(128)
        self.classifier = nn.Linear(128, num_classes)
    
    def forward(self, x):
        features = self.encoder(x)
        features = features.view(features.size(0), -1)
        features = self.bn(features)
        logits = self.classifier(features)
        return logits

class TripletLoss(nn.Module):
    def __init__(self, margin=0.3):
        super().__init__()
        self.margin = margin
    
    def forward(self, embeddings, labels):
        """
        Args:
            embeddings: [B, D] normalizados
            labels: [B] IDs de persona
        """
        dist_matrix = torch.cdist(embeddings, embeddings, p=2)
        batch_size = embeddings.size(0)
        loss = 0.0
        num_valid = 0
        
        for i in range(batch_size):
            pos_mask = (labels == labels[i]) & (torch.arange(batch_size, device=labels.device) != i)
            neg_mask = labels != labels[i]
            
            if not pos_mask.any() or not neg_mask.any():
                continue
            
            hard_positive = dist_matrix[i][pos_mask].max()
            hard_negative = dist_matrix[i][neg_mask].min()
            
            loss += F.relu(hard_positive - hard_negative + self.margin)
            num_valid += 1
        
        return loss / max(num_valid, 1)

In [ ]:
def train_supervised(save_path="supervised_model.pth"):
    print("ENTRENAMIENTO SUPERVISADO (CROSSENTROPY)")
    
    print("TRAIN SET:")
    train_dataset = CASIAB_Supervised(
        root_path=ROOT_PATH,
        subject_range=SUPERVISED_CONFIG['train_range'],
        conditions=SUPERVISED_CONFIG['conditions'],
        angles=SUPERVISED_CONFIG['angles'],
        frames_per_seq=SUBSET_FRAMES_PER_SEQ if USE_SUBSET else 30,
        img_size=IMG_SIZE,
        augment=True
    )
    
    print("\nVALIDATION SET:")
    val_dataset = CASIAB_Supervised(
        root_path=ROOT_PATH,
        subject_range=SUPERVISED_CONFIG['val_range'],
        conditions=SUPERVISED_CONFIG['conditions'],
        angles=SUPERVISED_CONFIG['angles'],
        frames_per_seq=SUBSET_FRAMES_PER_SEQ if USE_SUBSET else 30,
        img_size=IMG_SIZE,
        augment=False
    )
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    
    num_classes = train_dataset.get_num_classes()
    print(f"\nClases (personas) en train: {num_classes}\n")
    
    # Modelo cargando
    backbone = SSLBackbone(embed_dim=256)
    
    if os.path.exists(SSL_CHECKPOINT):
        checkpoint = torch.load(SSL_CHECKPOINT, map_location=DEVICE)
        backbone.load_state_dict(checkpoint['model_state_dict'])
        print(f"  ✓ Backbone SSL cargado: {SSL_CHECKPOINT}")
        print(f"    Epoch: {checkpoint['epoch']}, Loss: {checkpoint['loss']:.4f}")
    else:
        print(f"  ⚠ No se encontró {SSL_CHECKPOINT}")
        print(f"    Entrenando desde cero (no recomendado)")
    
    model = SupervisedReIDModel(
        backbone=backbone,
        num_classes=num_classes,
        freeze_backbone=False  # Entrenar todo el modelo
    ).to(DEVICE)
    
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"  Parámetros entrenables: {trainable:,} / {total:,} ({100*trainable/total:.1f}%)\n")
    
    # -------------------------------
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
    print(f"Configuración:")
    print(f"Epochs: {EPOCHS_SUPERVISED}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Steps/epoch: {len(train_loader)}\n")
    
    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(1, EPOCHS_SUPERVISED + 1):
        # TRAINING
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        start_time = time.time()
        
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            
            logits = model(images)
            loss = criterion(logits, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = logits.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        train_acc = 100. * train_correct / train_total
        
        # VALIDATION
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                logits = model(images)
                loss = criterion(logits, labels)
                
                val_loss += loss.item()
                _, predicted = logits.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_acc = 100. * val_correct / val_total
        
        scheduler.step()
        epoch_time = time.time() - start_time
        
        print(f"Epoch {epoch:2d}/{EPOCHS_SUPERVISED} | "
              f"Loss: {train_loss/len(train_loader):.3f} | "
              f"Train Acc: {train_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}% | "
              f"Time: {epoch_time:.1f}s")
        
        history['train_loss'].append(train_loss / len(train_loader))
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss / len(val_loader))
        history['val_acc'].append(val_acc)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'num_classes': num_classes,
                'history': history
            }, save_path)
    
    print(f"ENTRENAMIENTO COMPLETADO")
    print(f"Mejor Val Accuracy: {best_val_acc:.2f}%")
    print(f"Modelo guardado: {save_path}")
    
    return model, history

In [ ]:
model, history = train_supervised(save_path="supervised_model.pth")
    
print("\nHistorial de entrenamiento:")
print(f"Train Acc final: {history['train_acc'][-1]:.2f}%")
print(f"Val Acc final: {history['val_acc'][-1]:.2f}%")

ENTRENAMIENTO SUPERVISADO (CROSSENTROPY)
Cargando datasets...

TRAIN SET:
  Sujetos: 001-020 (20 sujetos)
  Condiciones: ['nm-01', 'nm-02']
  Ángulos: ['090']
  Samples: 787, Clases: 20

VALIDATION SET:
  Sujetos: 075-084 (10 sujetos)
  Condiciones: ['nm-01', 'nm-02']
  Ángulos: ['090']
  Samples: 400, Clases: 10

Clases (personas) en train: 20

Cargando modelo...

  ✓ Backbone SSL cargado: backbone_ssl_best.pth
    Epoch: 10, Loss: 3.0317
  Parámetros entrenables: 281,716 / 281,716 (100.0%)

⚙ Configuración:
  Epochs: 30
  Batch size: 32
  Learning rate: 0.0001
  Steps/epoch: 25

Epoch  1/30 | Loss: 2.970 | Train Acc: 9.02% | Val Acc: 1.25% | Time: 1.0s
Epoch  2/30 | Loss: 2.743 | Train Acc: 17.66% | Val Acc: 4.50% | Time: 0.9s
Epoch  3/30 | Loss: 2.571 | Train Acc: 24.90% | Val Acc: 6.50% | Time: 0.9s
Epoch  4/30 | Loss: 2.447 | Train Acc: 29.73% | Val Acc: 3.50% | Time: 0.9s
Epoch  5/30 | Loss: 2.306 | Train Acc: 36.09% | Val Acc: 3.25% | Time: 0.8s
Epoch  6/30 | Loss: 2.238 | Train

# Hybrid Phase

# Evaluación y testing